In [26]:
# %load database-reader.py
import sqlite3
import pandas as pd


Origin. 3040 raw match obersvations for English Premier League 
After merged with team arribute. it reduces to 2393 observations.


Missing Data: 
**1** from match table. we discard PS, SJ, GB, BS betting company daa + player featuures. focus on the Primary League only. 
**2** discard additional four lines with empty features 

In [102]:
db_location = "soccer/database.sqlite"

conn = sqlite3.connect(db_location)

In [75]:
df_match = pd.read_sql(sql="SELECT id,league_id, season, stage, date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, goal, shoton, shotoff, foulcommit, card, cross, corner, possession, B365H, B365D, B365A, BWH, BWD, BWA, IWH, IWD, IWA, LBH, LBD, LBA, WHH,WHD,WHA,VCH,VCD,VCA FROM Match WHERE league_id=1729 AND date >= '2010-02-22'", con=conn)
df_match=df_match.dropna()
df_match.head()


,id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,IWA,LBH,LBD,LBA,WHH,WHD,WHA,VCH,VCD,VCA
0,2228,1729,2009/2010,2,2010-03-24 00:00:00,658592,8456,8668,0,2,...,4.30,1.80,3.30,3.75,1.8,3.30,3.75,1.95,3.40,4.00
1,2239,1729,2009/2010,21,2010-03-16 00:00:00,658936,8528,10252,1,2,...,2.20,3.30,3.25,2.20,3.4,3.25,2.15,3.60,3.25,2.10
2,2240,1729,2009/2010,21,2010-03-23 00:00:00,658937,8654,8602,1,3,...,4.60,1.80,3.30,3.75,1.7,3.30,4.33,1.80,3.50,4.75
3,2245,1729,2009/2010,21,2010-03-10 00:00:00,658942,8191,10194,1,1,...,2.85,2.38,3.20,2.60,2.4,3.10,2.60,2.50,3.25,2.88
4,2246,1729,2009/2010,21,2010-03-09 00:00:00,658943,8472,8559,4,0,...,3.40,2.10,3.20,3.00,2.0,3.20,3.20,2.10,3.25,3.75


In [103]:
frames = list()

for index, row in df_match.iterrows():
    hteam=row.home_team_api_id
    ateam=row.away_team_api_id
    date=row.date ##match date
    hquery="SELECT * FROM team_attributes WHERE team_api_id="+str(hteam)
    aquery="SELECT * FROM team_attributes WHERE team_api_id="+str(ateam)
    df_hteam=pd.read_sql(sql=hquery,con=conn)
    df_ateam=pd.read_sql(sql=aquery,con=conn)
    df_hteam=df_hteam[df_hteam.date<=date]
    df_ateam=df_ateam[df_ateam.date<=date]
    df_hlastest= df_hteam.tail(1)
    df_alastest= df_ateam.tail(1)
    df_row=row.to_frame().T
    ##print df_row
    df_row_home=pd.merge(df_row, df_hlastest, how='outer',left_on='home_team_api_id', right_on='team_api_id', suffixes=('_match', '_home'))
    df_row_home_away=pd.merge(df_row_home,df_alastest, how='outer',left_on='away_team_api_id', right_on='team_api_id', suffixes=('_home', '_away'))
    frames.append(df_row_home_away)

    ##hteam_frames.append(df_hlastest)
    ##ateam_frames.append(df_alastest)

df_match_team = pd.concat(frames)
#df_ateams = pd.concat(ateam_frames)
df_match_team.head()


,id_match,league_id,season,stage,date_match,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,chanceCreationShooting_away,chanceCreationShootingClass_away,chanceCreationPositioningClass_away,defencePressure_away,defencePressureClass_away,defenceAggression_away,defenceAggressionClass_away,defenceTeamWidth_away,defenceTeamWidthClass_away,defenceDefenderLineClass_away
0,2228,1729,2009/2010,2,2010-03-24 00:00:00,658592,8456,8668,0,2,...,45,Normal,Organised,40,Medium,70,Double,40,Normal,Cover
0,2239,1729,2009/2010,21,2010-03-16 00:00:00,658936,8528,10252,1,2,...,50,Normal,Free Form,30,Deep,70,Double,30,Narrow,Cover
0,2240,1729,2009/2010,21,2010-03-23 00:00:00,658937,8654,8602,1,3,...,70,Lots,Organised,70,High,70,Double,70,Wide,Cover
0,2245,1729,2009/2010,21,2010-03-10 00:00:00,658942,8191,10194,1,1,...,55,Normal,Organised,35,Medium,70,Double,35,Normal,Cover
0,2246,1729,2009/2010,21,2010-03-09 00:00:00,658943,8472,8559,4,0,...,45,Normal,Organised,35,Medium,70,Double,35,Normal,Cover


In [107]:
df_match_team=df_match_team.rename(columns = [{'date':'date_away'},{'id':'id_away'}])
df_match_team.to_csv('Test.csv', index=None)

In [110]:
df_match_team.dtypes

id_match                                object
league_id                               object
season                                  object
stage                                   object
date_match                              object
match_api_id                            object
home_team_api_id                        object
away_team_api_id                        object
home_team_goal                          object
away_team_goal                          object
goal                                    object
shoton                                  object
shotoff                                 object
foulcommit                              object
card                                    object
cross                                   object
corner                                  object
possession                              object
B365H                                   object
B365D                                   object
B365A                                   object
BWH          

In [101]:
conn.close()